In [ ]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [ ]:
# not that we only call lv_tet because we will tetrahedralize it, we are loading a hex mesh!
lv_tet = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk") # load reference mesh (can also be done from other methods)
lv_tet.tetrahedralize(backend=TETRA_BACKEND.TETGEN, order=1, mindihedral=0.0, minratio=2.0) 
lv_tet.mesh

In [ ]:
lv_tet.identify_surfaces(
  endo_epi_args=dict(threshold=90.0),
  apex_base_args=dict(ab_ql=0.04, ab_qh=0.69),
  aortic_mitral_args=dict(a1=0.4,
                          a2=0.5,
                          a3=0.3,
                          a4=75,
                          a5=130,
                          
                          m1=0.17,
                          m2=0.02,
                          m3=0.07,
                          m4=0.333
                          )
)
lv_tet.plot("surface", scalars=LV_MESH_DATA.SURFS_DETAILED.value,
        cmap="tab20_r")

In [ ]:
lv_tet.compute_fibers(surfRegionsIds=LV_FIBER_MODES.LDRB_1,
                    fiber_space="P_1",
                    alpha_endo_lv=60,  # Fiber angle on the endocardium
                    alpha_epi_lv=-60,  # Fiber angle on the epicardium
                    beta_endo_lv=0,  # Sheet angle on the endocardium
                    beta_epi_lv=0,  # Sheet angle on the epicardium
                    markers={},
                    ldrb_kwargs={},
                    save_xdmfs=False,
                    xdmfs_dir=None,
                    xdmfs_basename=None,
                    del_generated_files=True,
                    )

In [ ]:
lv_tet.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.85)

In [ ]:
lv_hex = LV.from_pyvista_read("../sample_files/lvhexmesh.vtk") # reload hex mesh

In [ ]:
lv_hex.regress_fibers(lv_tet)

In [ ]:
lv_hex.plot_streamlines(LV_FIBERS.F0, scalars=LV_FIBERS.F0_ANGLES, decimate_boundary=0.85)